## Dependencies

In [1]:
import glob
import warnings
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Dense, Dropout, GlobalAveragePooling1D, SpatialDropout1D, Concatenate, Add
from googleqa_utilityscript import *
from transformers import BertConfig, BertTokenizer, TFBertModel


SEED = 0
seed_everything(SEED)
warnings.filterwarnings("ignore")

## Load data

In [2]:
BERT_PATH = '/kaggle/input/bert-base-uncased-huggingface-transformer/bert-base-uncased-tf_model.h5'
VOCAB_PATH = '/kaggle/input/bert-base-uncased-huggingface-transformer/bert-base-uncased-vocab.txt'
model_path_list = glob.glob('/kaggle/input/145-googleq-a-train-3fold-bert-base-unc-concat/' + '*1.h5')
model_path_list += glob.glob('/kaggle/input/145-googleq-a-train-3fold-bert-base-unc-concat/' + '*2.h5')
model_path_list += glob.glob('/kaggle/input/145-googleq-a-train-3fold-bert-base-unc-concat/' + '*3.h5')
model_path_list.sort()
print('Models to predict:')
print(*model_path_list, sep = "\n")

test = pd.read_csv('/kaggle/input/google-quest-challenge/test.csv')

print('Test samples: %s' % len(test))
display(test.head())

Models to predict:
/kaggle/input/145-googleq-a-train-3fold-bert-base-unc-concat/model_fold_1.h5
/kaggle/input/145-googleq-a-train-3fold-bert-base-unc-concat/model_fold_2.h5
/kaggle/input/145-googleq-a-train-3fold-bert-base-unc-concat/model_fold_3.h5
Test samples: 476


,qa_id,question_title,question_body,question_user_name,question_user_page,answer,answer_user_name,answer_user_page,url,category,host
0,39,Will leaving corpses lying around upset my pri...,I see questions/information online about how t...,Dylan,https://gaming.stackexchange.com/users/64471,There is no consequence for leaving corpses an...,Nelson868,https://gaming.stackexchange.com/users/97324,http://gaming.stackexchange.com/questions/1979...,CULTURE,gaming.stackexchange.com
1,46,Url link to feature image in the portfolio,I am new to Wordpress. i have issue with Featu...,Anu,https://wordpress.stackexchange.com/users/72927,I think it is possible with custom fields.\n\n...,Irina,https://wordpress.stackexchange.com/users/27233,http://wordpress.stackexchange.com/questions/1...,TECHNOLOGY,wordpress.stackexchange.com
2,70,"Is accuracy, recoil or bullet spread affected ...","To experiment I started a bot game, toggled in...",Konsta,https://gaming.stackexchange.com/users/37545,You do not have armour in the screenshots. Thi...,Damon Smithies,https://gaming.stackexchange.com/users/70641,http://gaming.stackexchange.com/questions/2154...,CULTURE,gaming.stackexchange.com
3,132,Suddenly got an I/O error from my external HDD,I have used my Raspberry Pi as a torrent-serve...,robbannn,https://raspberrypi.stackexchange.com/users/17341,Your Western Digital hard drive is disappearin...,HeatfanJohn,https://raspberrypi.stackexchange.com/users/1311,http://raspberrypi.stackexchange.com/questions...,TECHNOLOGY,raspberrypi.stackexchange.com
4,200,Passenger Name - Flight Booking Passenger only...,I have bought Delhi-London return flights for ...,Amit,https://travel.stackexchange.com/users/29089,I called two persons who work for Saudia (tick...,Nean Der Thal,https://travel.stackexchange.com/users/10051,http://travel.stackexchange.com/questions/4704...,CULTURE,travel.stackexchange.com


In [3]:
question_target_cols = ['question_asker_intent_understanding','question_body_critical', 'question_conversational', 
                        'question_expect_short_answer', 'question_fact_seeking', 'question_has_commonly_accepted_answer',
                        'question_interestingness_others', 'question_interestingness_self', 'question_multi_intent', 
                        'question_not_really_a_question', 'question_opinion_seeking', 'question_type_choice',
                        'question_type_compare', 'question_type_consequence', 'question_type_definition', 
                        'question_type_entity', 'question_type_instructions', 'question_type_procedure',
                        'question_type_reason_explanation', 'question_type_spelling', 'question_well_written']
answer_target_cols = ['answer_helpful', 'answer_level_of_information', 'answer_plausible', 'answer_relevance',
                      'answer_satisfaction', 'answer_type_instructions', 'answer_type_procedure', 
                      'answer_type_reason_explanation', 'answer_well_written']
target_cols = question_target_cols + answer_target_cols

## Pre-process data

In [4]:
text_features = ['question_title', 'question_body', 'answer']

# Model parameters

In [5]:
N_CLASS = len(target_cols)
MAX_SEQUENCE_LENGTH = 512

## Test set

In [6]:
tokenizer = BertTokenizer.from_pretrained(VOCAB_PATH, do_lower_case=True)

# Test features
X_test = compute_input_arays(test, text_features, tokenizer, MAX_SEQUENCE_LENGTH)

# Model

In [7]:
bert_config = BertConfig()
bert_config.output_hidden_states=True

def model_fn():
    input_word_ids = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_word_ids')
    input_masks = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_masks')
    segment_ids = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='segment_ids')

    bert_model = TFBertModel.from_pretrained(BERT_PATH, config=bert_config)
    sequence_output, pooled_output, hidden_states = bert_model([input_word_ids, input_masks, segment_ids])
    
    h12 = tf.reshape(hidden_states[-1][:, 0],(-1, 1, 768))
    h11 = tf.reshape(hidden_states[-2][:, 0],(-1, 1, 768))
    h10 = tf.reshape(hidden_states[-3][:, 0],(-1, 1, 768))
    h09 = tf.reshape(hidden_states[-4][:, 0],(-1, 1, 768))
    
    x = Concatenate(axis=1)([h12,h11, h10, h09])

    x = GlobalAveragePooling1D()(x)
    x = Dropout(0.2)(x)
    output = Dense(N_CLASS, activation="sigmoid", name="output")(x)

    model = Model(inputs=[input_word_ids, input_masks, segment_ids], outputs=output)
    
    return model

# Make predictions

In [8]:
Y_test = np.zeros((len(test), N_CLASS))

for model_path in model_path_list:
    model = model_fn()
    model.load_weights(model_path)
    Y_test += model.predict(X_test) / len(model_path_list)

In [9]:
submission = pd.read_csv('/kaggle/input/google-quest-challenge/sample_submission.csv')
submission[target_cols] = Y_test
submission.to_csv("submission.csv", index=False)
display(submission.head())
display(submission.describe())

,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
0,39,0.945672,0.643294,0.232442,0.490894,0.440092,0.459866,0.692071,0.696304,0.614262,...,0.915091,0.903643,0.623606,0.965388,0.954261,0.802105,0.048533,0.047590,0.865673,0.909182
1,46,0.897028,0.549231,0.007269,0.780370,0.803575,0.928944,0.562007,0.470501,0.061307,...,0.718286,0.938556,0.642093,0.973509,0.978380,0.891899,0.909707,0.117500,0.097257,0.891313
2,70,0.927353,0.632854,0.024816,0.775633,0.878344,0.943766,0.626826,0.493478,0.138169,...,0.866968,0.937412,0.608970,0.968180,0.971690,0.839319,0.085002,0.050703,0.889406,0.903135
3,132,0.904532,0.456511,0.009727,0.759090,0.725143,0.931594,0.546077,0.449650,0.043689,...,0.710105,0.952272,0.695315,0.970197,0.982746,0.899930,0.822119,0.156503,0.609547,0.893673
4,200,0.945210,0.427939,0.029746,0.863885,0.736725,0.869345,0.678987,0.589429,0.071364,...,0.701939,0.879981,0.630372,0.971265,0.965432,0.821455,0.258466,0.088290,0.690008,0.900196


,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
count,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,...,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000
mean,5029.186975,0.907041,0.600711,0.041399,0.750102,0.811254,0.858433,0.588611,0.493359,0.210931,...,0.813321,0.936258,0.663559,0.967830,0.973527,0.871027,0.516540,0.129191,0.529279,0.909516
std,2812.670060,0.035327,0.130590,0.079797,0.097259,0.105788,0.130097,0.053477,0.086551,0.197992,...,0.079257,0.023248,0.048785,0.010127,0.010994,0.041647,0.328658,0.061173,0.289453,0.020340
min,39.000000,0.778803,0.350140,0.003876,0.221596,0.225958,0.240665,0.480207,0.341134,0.008406,...,0.634077,0.793901,0.517842,0.922293,0.913627,0.691082,0.005636,0.009945,0.024149,0.812203
25%,2572.000000,0.884604,0.481999,0.007995,0.702762,0.772590,0.839240,0.546637,0.434854,0.062180,...,0.746910,0.927645,0.630311,0.963275,0.969384,0.850681,0.170328,0.084641,0.270738,0.898540
50%,5093.000000,0.907896,0.598369,0.012158,0.759463,0.822150,0.908097,0.579891,0.470836,0.124581,...,0.813645,0.942043,0.665335,0.970069,0.976110,0.879221,0.596010,0.131829,0.558233,0.911603
75%,7482.000000,0.933150,0.701677,0.028872,0.808686,0.876587,0.936587,0.624396,0.535649,0.312282,...,0.886019,0.951811,0.698236,0.974841,0.981291,0.899975,0.824201,0.169411,0.790353,0.923323
max,9640.000000,0.974739,0.885181,0.709950,0.956177,0.980593,0.974720,0.754930,0.768899,0.877516,...,0.951572,0.976026,0.783638,0.985861,0.992086,0.959701,0.958999,0.333142,0.988506,0.954134
